In [30]:
import os
import openmm
os.chdir('/Users/kendalllemons/Downloads/')

In [31]:
# This script was generated by OpenMM-Setup on 2023-01-17.

from openmm import *
from openmm.app import *
from openmm.unit import *

In [32]:
# Input Files

pdb = PDBFile('prepped_receptor_0-processed.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# System Configuration

nonbondedMethod = CutoffPeriodic
nonbondedCutoff = 1.0*nanometers
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
hydrogenMass = 1.5*amu

In [33]:
# Integration Options

dt = 0.005*picoseconds
temperature = 300*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 25

# Simulation Options

steps = 1000000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('Reference')
dcdReporter = DCDReporter('trajectory.dcd', 10000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 10000)

In [38]:
# Prepare the Simulation

print('Building system...')
topology = pdb.topology
positions = pdb.positions
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, boxSize=Vec3(5.0, 3.5, 3.5)*nanometers, model='tip3p')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

Building system...


In [ ]:
# Minimize and Equilibrate

print('Performing energy minimization...')
simulation.minimizeEnergy()
print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

In [ ]:
# Simulate

print('Simulating...')
simulation.reporters.append(PDBReporter('output(36).pdb', 10))
simulation.reporters.append(DCDReporter('output(36).dcd', 10))
simulation.currentStep = 0
simulation.step(steps)

In [ ]:
print('Saving...')
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('output(37).pdb', 'w'))
simulation.saveState('output.xml')
print('Done')